In [1]:
import mlflow
from mlflow.models import infer_signature
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso, LinearRegression, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import shap
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd

from pathlib import Path

/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
experiment_name = "house_prices"

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

mlflow.set_experiment(experiment_name)

train = pd.read_csv("data/train.csv")

y = np.log(train["SalePrice"])
X = train.drop(columns = ["Id", "SalePrice"])

X_train, X_val ,y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
num_cols = X.select_dtypes(include = ["number"]).columns
cat_cols = X.select_dtypes(include = ["object"]).columns


In [4]:
special_num_cols = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
other_num_cols = [col for col in num_cols if col not in special_num_cols]

masvnr_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value=0))
])

lotfront_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

def garageyr_imputer(X):
    X_filled = X.copy()
    median_year = X['YearBuilt'].median()
    X_filled['GarageYrBlt'] = X_filled['GarageYrBlt'].fillna(median_year)
    return X_filled

garage_tf = Pipeline([
    ("imputer", FunctionTransformer(garageyr_imputer))
])

# Pipeline numérique autres colonnes
other_num_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

# Pipeline catégoriel
cat_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Combiner tout
preprocess = ColumnTransformer([
    ("lotfront", lotfront_tf, ['LotFrontage']),
    ("masvnr", masvnr_tf, ['MasVnrArea']),
    ("garage", garage_tf, ['GarageYrBlt', 'YearBuilt']),  # On passe YearBuilt aussi pour l'imputer
    ("other_num", other_num_tf, other_num_cols),
    ("cat", cat_tf, cat_cols)
])


In [ ]:

models = {
    "random_forest": RandomForestRegressor(),
    "xgboost": XGBRegressor(),
    "lightgbm": LGBMRegressor(),
    "catboost": CatBoostRegressor(verbose=0),
    "lasso": Lasso(),
    "linear_regression": LinearRegression(),
    "ridge": RidgeCV(),
    "mlp": MLPRegressor(max_iter=1000)
}

# -------------------------
# Split dataset
# -------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

eval_data = pd.DataFrame(X_test, columns=X.columns)
eval_data["SalePrice"] = y_test

results = {}
models = {
    "random_forest": RandomForestRegressor(),
    "xgboost": XGBRegressor(),
    "lightgbm": LGBMRegressor(),
    "catboost": CatBoostRegressor(),
    "lasso" : Lasso(),
    "linear_regression": LinearRegression(),
    "ridge": RidgeCV(),
    "mlp": MLPRegressor()
}

results = {}
eval_data = X_val.copy()
eval_data["SalePrice"] = y_val

# -------------------------
# Boucle d'entraînement et log MLflow
# -------------------------
for model_name, model in models.items():
    with mlflow.start_run(run_name=f"eval_{model_name}_fe"):
        # Pipeline complet
        pipeline = Pipeline([
            ("preprocess", preprocess),
            ("model", model)
        ])
        
        pipeline.fit(X_train, y_train)
        
        # Exemple d'entrée pour MLflow
        input_example = X_train.sample(3)
        
        # Inférer la signature
        signature = infer_signature(input_example, pipeline.predict(input_example))
        
        # Log le modèle
        mlflow.sklearn.log_model(
            pipeline, 
            name="model",  # <-- utiliser name
            signature=signature,
            input_example=input_example
        )
        model_uri = f"runs:/{mlflow.active_run().info.run_id}/model"
        
        # Évaluation
        result = mlflow.models.evaluate(
            model_uri,
            eval_data,
            targets="SalePrice",
            model_type="regressor"
        )
        
        # Stocker les métriques
        results[model_name] = {
            "r2_score": result.metrics["r2_score"],
            "rmse": result.metrics["root_mean_squared_error"],
            "mae": result.metrics["mean_absolute_error"]
        }

# -------------------------
# Comparaison
# -------------------------
comparison_df = pd.DataFrame(results).T
print("Model Comparison:")
print(comparison_df[["r2_score", "rmse", "mae"]].round(3))


/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing v

🏃 View run eval_random_forest at: http://127.0.0.1:8080/#/experiments/1/runs/9b4e6c0d955346cbb73d59972847a728
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1


/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing v

🏃 View run eval_xgboost at: http://127.0.0.1:8080/#/experiments/1/runs/ced5b361d7e446e2be14c2151bb5fdf1
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005432 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3283
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 179
[LightGBM] [Info] Start training from score 12.030652


/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings

...

🏃 View run eval_linear_regression at: http://127.0.0.1:8080/#/experiments/1/runs/2ec00090ea184c338254fc707b761ade
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1


/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing v

🏃 View run eval_ridge at: http://127.0.0.1:8080/#/experiments/1/runs/7823f5093ebb462a8992b251293ca88f
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1


/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/media/sda1/repositories/house-prices/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missin

🏃 View run eval_mlp at: http://127.0.0.1:8080/#/experiments/1/runs/35a874ebbc074a0b91810d9c0ee1176c
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/1
Model Comparison:
                   r2_score   rmse    mae
random_forest         0.885  0.146  0.098
xgboost               0.880  0.149  0.099
lightgbm              0.895  0.140  0.091
catboost              0.911  0.129  0.085
lasso                 0.802  0.192  0.127
linear_regression     0.897  0.139  0.098
ridge               -38.460  2.714  1.668
mlp                -146.029  5.238  3.515
